In [25]:
.libPaths(R.home('/opt/homebrew/Caskroom/miniforge/base/lib/R/library'))
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.4. Data Wrangling/Output'
cur_date = '081125'

library(readxl)
library(tidyverse)
library(reshape2)
library(lubridate)
library(oce)
library(tigris)

# reading in files
ohio_tmax_2020_df = read_csv("Input/Ohio_Tmax_2020.csv")
ohio_tmax_2021_df = read_csv('Input/Ohio_Tmax_2021.csv')
ohio_tmax_2022_df = read_csv('Input/Ohio_Tmax_2022.csv')
ohio_PM_2020_df = read_csv('Input/Ohio_PM_2020.csv')
ohio_PM_2021_df = read_csv('Input/Ohio_PM_2021.csv')
ohio_PM_2022_df = read_csv('Input/Ohio_PM_2022.csv')
ohio_tmin_2020_df = read_csv('Input/Ohio_Tmin_2020.csv')
ohio_tmin_2021_df = read_csv('Input/Ohio_Tmin_2021.csv')
ohio_tmin_2022_df = read_csv('Input/Ohio_Tmin_2022.csv')
ohio_HI_2020_df = read_csv('Input/Ohio_HI_2020.csv')
ohio_HI_2021_df = read_csv('Input/Ohio_HI_2021.csv')
ohio_HI_2022_df = read_csv('Input/Ohio_HI_2022.csv')

To enable caching of data, set `options(tigris_use_cache = TRUE)`
in your R script or .Rprofile.

Rows: 1048575 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): DOY, x, y, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1048575 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): DOY, x, y, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1048575 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): DOY, x, y, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this messa

In [26]:
head(ohio_tmax_2020_df)

DOY,x,y,Value
<dbl>,<dbl>,<dbl>,<dbl>
50,2032326,4815818,76.1
50,2033252,4815818,76.1
50,2030473,4814892,75.9
50,2031399,4814892,76.9
50,2032326,4814892,76.5
50,2033252,4814892,76.5


In [44]:
convert_date = function(df, collection_year, variable){
    # converting the day of the year into a date with the month, day, and year
    
    new_df = df %>%
        mutate(Month = month(as_date(DOY)), Day = mday(as_date(DOY)), Year = collection_year,
           Date = format(make_date(month = Month, day = Day, year = Year), format = "%m/%d/%Y"),
           # going ahead and adding in the variable's name
           VariableName = variable) %>%
        select(-c('DOY', "Month", "Day", "Year")) 

    return(new_df)
    }

# calling fn
converted_tmax_2020_df = convert_date(ohio_tmax_2020_df, 2020, 'Maximum Temperature')
converted_tmax_2021_df = convert_date(ohio_tmax_2021_df, 2021, 'Maximum Temperature')
converted_tmax_2022_df = convert_date(ohio_tmax_2022_df, 2022, 'Maximum Temperature')
converted_HI_2020_df = convert_date(ohio_HI_2020_df, 2020, 'Heat Index')
converted_HI_2021_df = convert_date(ohio_HI_2021_df, 2021, 'Heat Index')
converted_HI_2022_df = convert_date(ohio_HI_2022_df, 2022, 'Heat Index')
converted_tmin_2020_df = convert_date(ohio_tmin_2020_df, 2020, 'Minimum Temperature')
converted_tmin_2021_df = convert_date(ohio_tmin_2021_df, 2021, 'Minimum Temperature')
converted_tmin_2022_df = convert_date(ohio_tmin_2022_df, 2022, 'Minimum Temperature')
# moving around these col names here to combine dfs below
converted_PM_2020_df = convert_date(ohio_PM_2020_df, 2020, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2021_df = convert_date(ohio_PM_2021_df, 2021, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)
converted_PM_2022_df = convert_date(ohio_PM_2022_df, 2022, 'PM2.5') %>%
    rename(Latitude = y, Longitude = x) %>%
    relocate(c("Latitude", "Longitude"), .after = VariableName)

head(converted_tmax_2020_df)

x,y,Value,Date,VariableName
<dbl>,<dbl>,<dbl>,<chr>,<chr>
2032326,4815818,76.1,02/20/2020,Maximum Temperature
2033252,4815818,76.1,02/20/2020,Maximum Temperature
2030473,4814892,75.9,02/20/2020,Maximum Temperature
2031399,4814892,76.9,02/20/2020,Maximum Temperature
2032326,4814892,76.5,02/20/2020,Maximum Temperature
2033252,4814892,76.5,02/20/2020,Maximum Temperature


In [45]:
convert_coordinates = function(df){
    # converting utm coordinates to latitude and longitude

    coordinates_df = df %>%
        # zone 14 is for Ohio
        mutate(Latitude = utm2lonlat(easting = x, northing = y, zone = 14)[[1]],
              Longitude = utm2lonlat(easting = x, northing = y, zone = 14)[[2]]) %>%
        select(-c('x', 'y')) 
    
    return(coordinates_df)
}

# calling fn
final_tmax_2020_df = convert_coordinates(converted_tmax_2020_df)
final_tmax_2021_df = convert_coordinates(converted_tmax_2021_df)
final_tmax_2022_df = convert_coordinates(converted_tmax_2022_df)
final_HI_2020_df = convert_coordinates(converted_HI_2020_df)
final_HI_2021_df = convert_coordinates(converted_HI_2021_df)
final_HI_2022_df = convert_coordinates(converted_HI_2022_df)
final_tmin_2020_df = convert_coordinates(converted_tmin_2020_df)
final_tmin_2021_df = convert_coordinates(converted_tmin_2021_df)
final_tmin_2022_df = convert_coordinates(converted_tmin_2022_df)

head(final_tmax_2020_df)

Value,Date,VariableName,Latitude,Longitude
<dbl>,<chr>,<chr>,<dbl>,<dbl>
76.1,02/20/2020,Maximum Temperature,-80.53294,41.98001
76.1,02/20/2020,Maximum Temperature,-80.52234,41.97824
75.9,02/20/2020,Maximum Temperature,-80.55653,41.97562
76.9,02/20/2020,Maximum Temperature,-80.54592,41.97386
76.5,02/20/2020,Maximum Temperature,-80.53531,41.97209
76.5,02/20/2020,Maximum Temperature,-80.52471,41.97032


Now breaking these data up into the dataframes that they will be stored in for the database. We'll have separate dataframes for each year. 

In [46]:
twenty20_df = rbind(converted_PM_2020_df, final_HI_2020_df, final_tmax_2020_df, final_tmin_2020_df)
twenty21_df = rbind(converted_PM_2021_df, final_HI_2021_df, final_tmax_2021_df, final_tmin_2021_df)
twenty22_df = rbind(converted_PM_2022_df, final_HI_2022_df, final_tmax_2022_df, final_tmin_2022_df)

In [47]:
# adding in georaphic id and variable ids
split_dfs = function(df){
    new_df = df %>%
        mutate(Num = 1:n(), GeographyID = paste0('GEO', Num), ValueID = paste0('Val', Num),
              VariableID = ifelse(VariableName == "PM2.5", 'PM',
                                      ifelse(VariableName == "Maximum Temperature",'MaxT',
                                          ifelse(VariableName == "Minimum Temperature", 'MinT',
                                              ifelse(VariableName == "Heat Index", 'HI', NA))))) %>%
        select(-Num)

    # creating the geography df
    geo_df = unique(new_df[,c(6,4,5)])

    # creating the variable df
    var_df = unique(new_df[,c(8,3)])

    # value df
    value_df = new_df[,c(7,8,1)]

    # health data df
    health_df = new_df[,c(2,7)]

    return(list(geo_df, var_df, value_df, health_df))
}

# calling fn
split_twenty20_dfs = split_dfs(twenty20_df)
split_twenty21_dfs = split_dfs(twenty21_df)
split_twenty22_dfs = split_dfs(twenty22_df)

In [48]:
# viewing
head(split_twenty20_dfs[[1]])
head(split_twenty20_dfs[[2]])
head(split_twenty20_dfs[[3]])
head(split_twenty20_dfs[[4]])

GeographyID,Latitude,Longitude
<chr>,<dbl>,<dbl>
GEO1,41.9661,-80.585
GEO2,41.9661,-80.555
GEO3,41.9661,-80.525
GEO4,41.9361,-80.675
GEO5,41.9361,-80.645
GEO6,41.9361,-80.615


VariableID,VariableName
<chr>,<chr>
PM,PM2.5
HI,Heat Index
MaxT,Maximum Temperature
MinT,Minimum Temperature


ValueID,VariableID,Value
<chr>,<chr>,<dbl>
Val1,PM,12.0
Val2,PM,12.0
Val3,PM,11.9
Val4,PM,12.2
Val5,PM,12.2
Val6,PM,12.1


Date,ValueID
<chr>,<chr>
07/08/2020,Val1
07/08/2020,Val2
07/08/2020,Val3
07/08/2020,Val4
07/08/2020,Val5
07/08/2020,Val6


Adding in additional geography data like census tract, county name etc. to the geography dataframes.

In [60]:
# download shapefile from the tigris (census package)
ohio_county_shapefile = data.frame(tracts(state = "Ohio", year = 2020)[,c(2,4,5,11,12)]) %>%
    rename(FIPS = GEOID, CensusTract = NAME, Latitude = INTPTLON, Longitude = INTPTLAT) %>%
    select(-geometry) %>%
    mutate(Latitude = round(as.numeric(Latitude), 4), Longitude = round(as.numeric(Longitude), 3))

head(ohio_county_shapefile)

,COUNTYFP,FIPS,CensusTract,Longitude,Latitude
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,153,39153533102,5331.02,41.244,-81.4167
2,153,39153533200,5332,41.218,-81.4164
3,029,39029951600,9516,40.728,-80.5860
4,093,39093070600,706,41.379,-82.0924
5,093,39093097100,971,41.122,-82.2636
6,093,39093010200,102,41.507,-82.0124


In [62]:
geography_2020_df = split_twenty20_dfs[[1]]

left_join(head(geography_2020_df), ohio_county_shapefile)

Joining with `by = join_by(Latitude, Longitude)`


GeographyID,Latitude,Longitude,COUNTYFP,FIPS,CensusTract
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
GEO1,41.9661,-80.585,NA,NA,NA
GEO2,41.9661,-80.555,NA,NA,NA
GEO3,41.9661,-80.525,NA,NA,NA
GEO4,41.9361,-80.675,NA,NA,NA
GEO5,41.9361,-80.645,NA,NA,NA
GEO6,41.9361,-80.615,NA,NA,NA
